In [2]:
# Import statements
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import sys
from glob import glob
from pymatreader import read_mat
from scipy import signal
from scipy import stats
from scipy.ndimage import uniform_filter1d

import itertools

#import matlab.engine
# Test out matlab spike detection code
#eng = matlab.engine.start_matlab()
#eng.eval("startup", nargout=0)

# Importing custom python files
import consts
import importlib
importlib.reload(consts)

from open_ephys.analysis import Session

%matplotlib tk

f = os.sep

In [3]:
# Read in pickle file
interm_data_path =  f + 'home' +f+ 'pierfier' +f+ 'Projects' +f+ 'Pierre Fabris' +f+ 'PV DBS neocortex' +f+ 'Interm_Data' +f+ 'flicker.pkl'
df =pd.read_pickle(interm_data_path)
print(df.columns)

Index(['frame_time', 'raw_trace', 'sub_vm', 'detrend_vm', 'trace_noise',
       'interp_time', 'interp_raw_trace', 'interp_raw_detrend_trace',
       'interp_spike_raster', 'spike_amp_raster', 'flicker_raster',
       'stim_raster', 'interp_subvm', 'stim_freq', 'flicker_freq', 'mouse_id',
       'session_id', 'trial_id', 'stim_param', 'fov_id', 'roi_id'],
      dtype='object')


In [4]:
#TODO need to redo the spike detection???
save_path = f+ 'home' +f+ 'pierfier' +f+ 'Projects' +f+ 'Pierre Fabris' +f+ 'PV DBS neocortex' +f+ 'flicker_traces' +f

# Loop through each unique ROI and plot their traces with stimulation
test_freqs = df['stim_freq'].unique()
for freq in test_freqs:
    freq_df = df[df['stim_freq'] == freq]
    
    
    mouse_list = freq_df['mouse_id'].unique()
    session_list = freq_df['session_id'].unique()
    fov_list = freq_df['fov_id'].unique()

    pairings = list(itertools.product(mouse_list, session_list, fov_list))
    
    # Loop through unique FOVs
    for values in pairings:
        fov_df = freq_df[(freq_df['mouse_id'] == values[0]) & \
                         (freq_df['session_id'] == values[1]) & \
                            (freq_df['fov_id'] == values[2])]
        
        # Skip if dataframe is empty()
        if fov_df.empty:
            continue

        # Plot all trials in a single figure
        plt.figure(figsize=(12, 12))
        # Loop through each trial
        for tr_i, tr in enumerate(fov_df['trial_id'].unique()[:]):
            trial_df = fov_df[fov_df['trial_id'] == tr]
            
            # Construct dataframe of individual trials
            plot_df = trial_df.pivot(index=['trial_id'], columns='interp_time', values='interp_raw_detrend_trace')
            sp_df = trial_df.pivot(index=['trial_id'], columns='interp_time', values='interp_spike_raster')
            sp_idx = np.where(sp_df.values[0] == 1)
            
            timeline = plot_df.columns.values
            cur_vm = plot_df.values[0]

            plt.plot(timeline, tr_i*0.05 + cur_vm, '-b')
            plt.plot(timeline[sp_idx], tr_i*0.05 + cur_vm[sp_idx], '.r')
        plt.title(values[0] + '_' + values[1] + '_fov' + str(values[2]) + '_freq' + str(freq))
        plt.savefig(save_path + values[0] + '_' + values[1] + '_' + str(values[2]) +'_'+ str(freq) +'.png')
        plt.show()

/tmp/ipykernel_114832/2744242736.py:27: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(12, 12))


In [5]:
df_new = df.set_index(['mouse_id', 'session_id', 'fov_id', 'stim_freq'])

# Print the number of 40Hz
df_40 = df[df['stim_freq'] == 40]
df_140 = df[df['stim_freq'] == 140]


df_40 = df_40.set_index(['mouse_id', 'session_id', 'fov_id'])
df_140 = df_140.set_index(['mouse_id', 'session_id', 'fov_id'])

print(df_40.index.nunique())
print(df_140.index.nunique())


print(df[df['interp_time'] == 0])

df_temp = df[df['interp_time'] == 0]



df_pv = df_temp.pivot(index=['mouse_id', 'session_id', 'fov_id', 'stim_freq'], columns='trial_id', values='interp_raw_trace')
print('The pivoted dataframe')
print(df_pv)

print(df_pv.index.nunique())
print(df_new.index.nunique())

11
13
        frame_time   raw_trace    sub_vm  detrend_vm  trace_noise  \
14860          0.0  944.401299 -0.002507   -0.004159     0.003023   
19832          0.0  867.038745 -0.002884    0.000019     0.003957   
22318          0.0  835.242424 -0.002913   -0.001171     0.002961   
29776          0.0  764.839610 -0.004883   -0.004883     0.004812   
42206          0.0  773.934632 -0.000995   -0.001487     0.002985   
...            ...         ...       ...         ...          ...   
934680         0.0  888.577394  0.019306    0.017184     0.003493   
937166         0.0  851.909669 -0.011914   -0.012788     0.003575   
939652         0.0  845.243759 -0.000509   -0.001770     0.003345   
942138         0.0  850.502951  0.010276    0.008596     0.002786   
944624         0.0  834.259646 -0.006658   -0.005264     0.003361   

        interp_time  interp_raw_trace  interp_raw_detrend_trace  \
14860           0.0        944.401299                 -0.004159   
19832           0.0        867.

In [6]:
print(df['mouse_id'].unique())

['109558_Vb_male', '109567_Vb_male']
Categories (2, object): ['109558_Vb_male', '109567_Vb_male']


In [7]:
plt.close('all')